# <div align="center" > The custom environment for the Project Portfolio simulation </div>
<div align="center" > This is the simulation notebook for the project portfolio environement simulation.
</br>
</br>


---
--- 
# The prerequisites
<span style="padding-left: 2em;">This continues the paragraph normally.here we're going to elaborate on the problem definition and design choices.</span>


## Project management relative concepts
<span style="padding-left: 2em;">This continues the paragraph normally.here we're going to elaborate on the problem definition and design choices.</span>

### Project Cost Performance Baseline
![Chart](../Assets/Project-cost-baseline-graph.jpg)

---
---
## Problem deffinition

<span style="padding-left: 2em;">This continues the paragraph normally.here we're going to elaborate on the problem definition and design choices.</span>

## Inclusions and exclusions
* Contingency and management reserves
* Cashflow Uncertainties



### environment design 


---
---
# 🚀Initiation

In [ ]:
from gymnasium import Env
from gymnasium.spaces import Discrete, Boxs
import numpy as np



ImportError: cannot import name 'Boxs' from 'gymnasium.spaces' (f:\Career\01. IE (Industrial Engineering)\00. Univesity\Masters\05. Master's thesis\PPO-project-portfolio-budgeting-AI-agent\.venv\Lib\site-packages\gymnasium\spaces\__init__.py)